# Turkish ULMFiT from scratch

### this is my notebook for trying to feature engineer the medical history information into something useful

In [3]:
import pandas as pd
import numpy as np

import os
import re
from collections import Counter

In [3]:
#use this cell when working online
path = '/floyd/home/ed-triage'
data_path = '/floyd/home/data'

In [4]:
#use this cell when working from home
path = '/Users/jjaskolkambp/Desktop/machine learning/my_projects/ed-triage'
data_path = '/Users/jjaskolkambp/Desktop/machine learning/my_projects/data/ED triage project/combo'

In [5]:
clin = pd.read_csv(data_path + '/complete_clean_combo_data.csv', index_col = 0,low_memory = False)

In [6]:
clin.shape

(165433, 121)

In [7]:
data = clin['pmhx'].str.split(',')

In [8]:
history = list(clin['MedicalHistory'].str[16:].astype('str'))

In [9]:
#this is to clean the text in the medical history column (a little bit)
def fixhx(s):
    s =s.replace('<LT><LF>','')
    s = s.replace('<LT>LF>','')
    s =s.replace('Pt.', 'patient')
    s =s.replace('Pt', 'patient')
    s = s.lower()
    s = s.replace('pt.', 'patient')
    s = s.replace('y/o', 'year old')
    s = s.replace('c/o', 'complains of')
    s = s.replace('sob', 'shortness of breath')
    s = s.replace('c/p', 'chest pain')
    s = s.replace('hrs', 'hours')
    s = s.replace('hx', 'history')
    s = s.replace('n/v', 'nausea and/or vomiting')
    s = s.replace('a/e', 'air entry')
    s = s.replace('a/o', 'alert and oriented')
    s = s.replace('a&o', 'alert and oriented')
    s = s.replace('d/c', 'discharge')
    s = s.replace('u/s', 'ultrasound')
    s = s.replace('yrs', 'years')
    s = s.replace('lmp', 'last menstrual period')
    s = s.replace('w/', 'with')
    s = s.replace('@', 'at ')
    s = s.replace('b/c', 'because')
    s = re.sub(r'\bpt.','patient ',s)
    s = re.sub(r'\blt.','left ',s)
    s = re.sub(r'\brt.','right ',s)
    s = re.sub(r'\bmd.','doctor ',s)
    s = re.sub(r'\bra\b','room air ',s)
    s = re.sub(r'\bbp.','blood pressure ',s)
    s = re.sub(r'\bcp.','chest pain ',s)
    s = re.sub(r'\bfd.','family doctor ',s)
    s = s.replace('abx', 'antibiotics')
    s = s.replace(' htn ', ' hypertension ')
    s = re.sub(r'\btyl\b', 'tylenol', s)
    s = re.sub(r'\bprn\b', 'as needed ',s)
    s = re.sub(r'\bvag\b', 'vaginal ',s)
    s = re.sub(r'\bpo\b', 'by mouth ',s)
    s = re.sub(r'\bllq\b', 'left lower quadrant',s)
    s = re.sub(r'\brlq\b', 'right lower quadrant  ',s)
    s = re.sub(r'\bluq\b', 'left upper quadrant',s)
    s = re.sub(r'\bruq\b', 'right upper quadrant  ',s)
    s = re.sub(r'\buti\b', 'urinary tract infection ',s)
    s = re.sub(r'\brd', 'right digit number',s)
    s = re.sub(r'\bld', 'left digit number',s)
    s = re.sub(r'\br\b', 'right ',s)
    s = re.sub(r'\bl\b', 'left ',s)
    s = re.sub(r'\bed\b', 'emergency department ',s)
    s = re.sub(r'\ber\b', 'emergency room ',s)
    s = re.sub(r'\bbm\b', 'bowel movement ',s)
    s = re.sub(r'#\s', 'fracture ', s)
    
    #remove extra white spaces
    s = re.sub(r'\s+', ' ', s)

    #note that what's missing here is removal of punctuation
    return s

In [10]:
replacements = (',','<LT>LF>', '.')
drop_list = ('"', '?')
split_hist = []
for item in history:
    for r in replacements:
        item = item.replace(r, '::')
    
    item = item.split('::')
    
    item = [re.sub(r'\(.*?\)\ *', '', s) for s in item]
    
    item = [fixhx(s) for s in item]
    for d in drop_list:
        item = [x.replace(d,'') for x in item]
    
    item = [x.strip().lower() for x in item if x != 'nan']
    item = [x for x in item if len(x) > 2]
    
    item = list(filter(None, item))
    
    split_hist.append(item)
    

In [11]:
#rejoining them as a string with diagnoses separated by commas
join_hist = [','.join(item) for item in split_hist]

In [12]:
comorbids = [len(item) if item != ['no significant medical history'] else 0 for item in split_hist]

In [13]:
#replacing empty string with NaN
join_hist = [np.nan if item == 'nan' else item for item in join_hist]

In [14]:
len(join_hist)

165433

In [15]:
join_hist[:20]

['no significant medical history',
 'no significant medical history',
 'no significant medical history',
 'dialysis haemo/pd,iddm,crf,heart problem,high cholesterol',
 'fibromyalgia',
 'enlarged liver',
 'anxiety,autistic,adhd,depression,asthma,celiac',
 'no significant medical history',
 'hypothyroid',
 'anxiety',
 'no significant medical history',
 'hypertension',
 'arthritis,copd',
 'itp,wiscott-aldridge syndrome,eczema',
 'suicidal attempts,depression',
 'high cholesterol,hypertension,niddm,cabg,left knee surgery 15 years ago',
 'sepsis,urinary tract infection,ca lungs,ca bronchus,hypothyroid,glaucoma cataract/cataract repair,kidney disease,anemia',
 'alcoholism',
 'gerd,high cholesterol',
 'hypertension,tia,osteoporosis,diverticulitis,high cholesterol,anxiety,depression,aortic stenosis']

In [90]:
split_hist[:20]

[['no significant medical history'],
 ['no significant medical history'],
 ['no significant medical history'],
 ['dialysis haemo/pd', 'iddm', 'crf', 'heart problem', 'high cholesterol'],
 ['fibromyalgia'],
 ['enlarged liver'],
 ['anxiety', 'autistic', 'adhd', 'depression', 'asthma', 'celiac'],
 ['no significant medical history'],
 ['hypothyroid'],
 ['anxiety'],
 ['no significant medical history'],
 ['hypertension'],
 ['arthritis', 'copd'],
 ['itp', 'wiscott-aldridge syndrome', 'eczema'],
 ['suicidal attempts', 'depression'],
 ['high cholesterol',
  'hypertension',
  'niddm',
  'cabg',
  'left knee surgery 15 years ago'],
 ['sepsis',
  'urinary tract infection',
  'ca lungs',
  'ca bronchus',
  'hypothyroid',
  'glaucoma cataract/cataract repair',
  'kidney disease',
  'anemia'],
 ['alcoholism'],
 ['gerd', 'high cholesterol'],
 ['hypertension',
  'tia',
  'osteoporosis',
  'diverticulitis',
  'high cholesterol',
  'anxiety',
  'depression',
  'aortic stenosis']]

### exploring different diagnoses

In [16]:
diagnoses = Counter()
for hx in split_hist:
    for item in hx:
        diagnoses[item] += 1

In [17]:
#there are 23721 different medical histories used in the dataset!!!
len(diagnoses)

23721

In [18]:
#sorted diagnoses
diagnoses.most_common(40)

[('no significant medical history', 73713),
 ('hypertension', 21703),
 ('high cholesterol', 19767),
 ('niddm', 12394),
 ('htn', 7693),
 ('asthma', 6974),
 ('high bp', 6399),
 ('depression', 6375),
 ('thyroid', 4936),
 ('anxiety', 4855),
 ('hypothyroid', 3981),
 ('iddm', 3800),
 ('gerd', 3409),
 ('diabetes', 3257),
 ('acid reflux', 2420),
 ('arthritis', 2220),
 ('anemia', 1706),
 ('copd', 1511),
 ('high chol', 1283),
 ('chf', 1172),
 ('schizophrenia', 1154),
 ('gout', 1120),
 ('seizure', 1057),
 ('kidney stones', 1021),
 ('enlarged prostate', 1016),
 ('cva', 1011),
 ('atrial fibrillation', 952),
 ('dementia', 924),
 ('cholesterol', 893),
 ('migraines', 874),
 ('adhd', 794),
 ('osteoporosis', 784),
 ('bipolar', 723),
 ('chronic back pain', 717),
 ('afib', 716),
 ('immunizations up to date', 688),
 ('smoker', 675),
 ('cad', 624),
 ('fibromyalgia', 601),
 ('substance misuse', 597)]

In [19]:
diagnoses.most_common()[-40:]

[('tear in aorta', 1),
 ('vertebra fracture', 1),
 ('ulcers both feet', 1),
 ('wallerian degenration', 1),
 ('hemiplegia at birth', 1),
 ('ventricular enlargement', 1),
 ('myocaridial infarction five years ago', 1),
 ('stroke fifteen years ago', 1),
 ('bilateral lung transplant', 1),
 ('copa syndrome', 1),
 ('cellulitis developed from c-section', 1),
 ('low hg-blood transfusion 3months ago', 1),
 ('back injury -high chol', 1),
 ('hashimotos-auto immune ds', 1),
 ('prolactin level high-', 1),
 ('stent-heart blockage', 1),
 ('torn ligament of left leg', 1),
 ('bilateral lower leg amputations', 1),
 ('currently being assessed for delirium', 1),
 ('stent 9 months ago', 1),
 ('legs paralyed', 1),
 ('5 tetnts 2018', 1),
 ('carotid sx', 1),
 ('demyleanating neuropathy', 1),
 ('paresthesia', 1),
 ('type ii diabetes', 1),
 ('unspecified heart disease', 1),
 ('list', 1),
 ('c6 stenosis', 1),
 ('left femur fracture three years ago', 1),
 ('abaltion st', 1),
 ('mikes on march 26', 1),
 ('select fr

In [20]:
#this turns the top 1000 classes into a dict where each class has a number
class_dict = {finding[0]:i for i,finding in enumerate(diagnoses.most_common(999))}; class_dict

{'no significant medical history': 0,
 'hypertension': 1,
 'high cholesterol': 2,
 'niddm': 3,
 'htn': 4,
 'asthma': 5,
 'high bp': 6,
 'depression': 7,
 'thyroid': 8,
 'anxiety': 9,
 'hypothyroid': 10,
 'iddm': 11,
 'gerd': 12,
 'diabetes': 13,
 'acid reflux': 14,
 'arthritis': 15,
 'anemia': 16,
 'copd': 17,
 'high chol': 18,
 'chf': 19,
 'schizophrenia': 20,
 'gout': 21,
 'seizure': 22,
 'kidney stones': 23,
 'enlarged prostate': 24,
 'cva': 25,
 'atrial fibrillation': 26,
 'dementia': 27,
 'cholesterol': 28,
 'migraines': 29,
 'adhd': 30,
 'osteoporosis': 31,
 'bipolar': 32,
 'chronic back pain': 33,
 'afib': 34,
 'immunizations up to date': 35,
 'smoker': 36,
 'cad': 37,
 'fibromyalgia': 38,
 'substance misuse': 39,
 'urinary tract infection': 40,
 'pneumonia': 41,
 'chol': 42,
 'pacemaker/cardioverter/defib': 43,
 'breast cancer': 44,
 'cabg': 45,
 'low iron': 46,
 'eczema': 47,
 'sleep apnea': 48,
 'rheumatoid arthritis': 49,
 'blood pressure': 50,
 'bi-polar': 51,
 'stroke': 52

In [21]:
#this creates a large array containing a 1 in each column where there is a diagnosis
complete_findings = np.empty((len(data), 1000))
for idx,findings in enumerate(split_hist):
    findings_array = np.zeros(1000)
    for finding in findings:
        try:
            findings_array[class_dict[finding]] = 1
        except:
            findings_array[999] = 1
    complete_findings[idx,:]= findings_array

In [22]:
#this turns the above array into a dataframe
findings_df = pd.DataFrame(complete_findings.astype(int), columns = list(class_dict.keys()) + ['other']); findings_df.head(10)

,no significant medical history,hypertension,high cholesterol,niddm,htn,asthma,high bp,depression,thyroid,anxiety,...,lactose intolerance,pinched nerve,frequent utis,ranitidine,lantus,ativan,migrains,pih,ataxia,other
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
7,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [23]:
findings_df.shape

(165433, 1000)

In [24]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from fastai import *
from fastai.text import *

In [26]:
mddf = pd.DataFrame(join_hist)

In [33]:
data = (TextList.from_df(mddf)
        .split_by_rand_pct(0.1)
        .label_for_lm()
        .databunch(bs=bs, num_workers=1))

data.save('medhx_databunch')
len(data.vocab.itos),len(data.train_ds)

(3056, 148890)

In [37]:
data.vocab.itos[100:150]

['fibromyalgia',
 'with',
 'removed',
 'valve',
 'issues',
 'cabg',
 'sleep',
 "'s",
 'abuse',
 'of',
 'defib',
 'cholestrol',
 'cardioverter',
 'mi',
 'apnea',
 'dialysis',
 'patient',
 'glaucoma',
 'eczema',
 'injury',
 'c',
 'cyst',
 'bi',
 'etoh',
 'weeks',
 'angioplasty',
 'colitis',
 'sx',
 'rheumatoid',
 'a',
 'colon',
 'brain',
 'hip',
 'polar',
 'ovarian',
 'd',
 'at',
 'cataract',
 'leg',
 'stomach',
 'syndrome',
 'hysterectomy',
 'epilepsy',
 'failure',
 'dvt',
 'cell',
 'diverticulitis',
 'on',
 'renal',
 'ibs']

In [36]:
data.show_batch()

idx,text
0,"eczema xxbos suicidal attempts , depression xxbos high cholesterol , hypertension , niddm , cabg , left knee surgery 15 years ago xxbos sepsis , urinary tract infection , ca lungs , ca xxunk , hypothyroid , glaucoma cataract / cataract repair , kidney disease , anemia xxbos alcoholism xxbos gerd , high cholesterol xxbos hypertension , tia , osteoporosis , diverticulitis , high cholesterol , anxiety , depression ,"
1,"fibromyalgia , copd , asthma , cad , cad xxbos hypertension , high cholesterol , niddm , anemia xxbos no significant medical history xxbos no significant medical history xxbos sleep apnea , asthma xxbos high cholesterol xxbos hyperthyroid , high cholesterol , atrial fibrillation xxbos no significant medical history xxbos hole in heart xxbos no significant medical history xxbos no significant medical history xxbos alzheimer's , hypertension , bipolar ,"
2,"bp , high cholesterol , anticoagulants , niddm xxbos no significant medical history xxbos high bp , high cholesterol , diabetes xxbos anemia xxbos no significant medical history xxbos acne , depression xxbos patient d , bi - polar xxbos high cholesterol , hypertension xxbos high bp xxbos no significant medical history xxbos no significant medical history xxbos migraines , pneumonia xxbos hypertension xxbos no significant medical history xxbos angioplasty"
3,"no significant medical history xxbos bipolar , schizophrenia xxbos no significant medical history xxbos no significant medical history xxbos hypertension , high cholesterol , niddm xxbos pancreatitis xxbos no significant medical history xxbos no significant medical history xxbos substance misuse xxbos gerd xxbos tumor to adrenal glands and pancreas , hypertension , copd , diverticulitis xxbos lung cancer xxbos no significant medical history xxbos niddm , high cholesterol , high"
4,"no significant medical history xxbos immunizations up to date , vit d deficiency xxbos no significant medical history xxbos no significant medical history xxbos no significant medical history xxbos hypertension xxbos no significant medical history xxbos no significant medical history xxbos asthma xxbos no significant medical history xxbos no significant medical history xxbos spinal cord injury , emphysema , copd , indwelling catheter xxbos no significant medical history xxbos no"


In [13]:
learn = language_model_learner(data, AWD_LSTM, drop_mult=0.1, wd=0.1, pretrained=False).to_fp16()

In [14]:
lr = 3e-3
lr *= bs/48  # Scale learning rate by batch size

In [15]:
learn.unfreeze()
learn.fit_one_cycle(10, lr, moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,time
0,4.337346,4.472464,0.297502,10:04
1,4.145160,4.359449,0.304195,10:29
2,4.199796,4.386935,0.301542,10:29
3,4.132432,4.329662,0.305723,10:30
4,4.048324,4.241048,0.313223,10:32
5,3.943493,4.118834,0.323640,10:30
6,3.857112,3.969563,0.337475,10:30
7,3.695721,3.799551,0.354562,10:30
8,3.602813,3.652005,0.371855,10:31
9,3.519019,3.596706,0.379076,10:31


In [32]:
learn.to_fp32().save(lm_fns[0], with_opt=False)
learn.data.vocab.save(lm_fns[1].with_suffix('.pkl'))

## Turkish sentiment analysis

https://www.win.tue.nl/~mpechen/projects/smm/

### Language model

In [24]:
path_clas = path/'movies'
path_clas.ls()

[PosixPath('/home/sgugger/.fastai/data/trwiki/movies/tr_polarity.pos'),
 PosixPath('/home/sgugger/.fastai/data/trwiki/movies/tr_clas_databunch'),
 PosixPath('/home/sgugger/.fastai/data/trwiki/movies/models'),
 PosixPath('/home/sgugger/.fastai/data/trwiki/movies/tr_polarity.neg'),
 PosixPath('/home/sgugger/.fastai/data/trwiki/movies/tr_data_lm')]

In [25]:
pos = (path_clas/'tr_polarity.pos').open(encoding='iso-8859-9').readlines()
pos_df = pd.DataFrame({'text':pos})
pos_df['pos'] = 1
pos_df.head()

,text,pos
0,gerçekten harika bir yapim birçok kez izledim ...,1
1,her izledigimde hayranlik duydugum gerçek klas...,1
2,gerçekten tarihi savas filmleri arasinda tarti...,1
3,aldigi ödülleri sonuna dek hak eden muhtesem b...,1
4,özgürlük denilince aklima gelen ilk film.bir b...,1


In [26]:
neg = (path_clas/'tr_polarity.neg').open(encoding='iso-8859-9').readlines()
neg_df = pd.DataFrame({'text':neg})
neg_df['pos'] = 0
neg_df.head()

,text,pos
0,giseye oynayan bir film.mel gibson'in oyunculu...,0
1,bircok yonden sahip olduklari zayifliklari pop...,0
2,"1995 ten bu yana bu tür filmler artti , o zama...",0
3,mel gibson tam bir ingiliz düsmani her filmind...,0
4,milliyetçi bir film tavsiye etmiyorum.... \n,0


In [27]:
df = pd.concat([pos_df,neg_df], sort=False)

In [28]:
data_lm = (TextList.from_df(df, path_clas, cols='text', processor=SPProcessor.load(dest))
    .split_by_rand_pct(0.1, seed=42)
    .label_for_lm()           
    .databunch(bs=bs, num_workers=1))

data_lm.save(f'{lang}_clas_databunch')

In [29]:
data_lm = load_data(path_clas, f'{lang}_clas_databunch', bs=bs)

In [30]:
data_lm.show_batch()

idx,text
0,"▁bile ▁sinema ▁olgusu nun ▁en ▁üst ▁noktalar in dan . . ▁xxbos ▁gerçekten ▁tarihi ▁sava s ▁filmleri ▁ara si nda ▁tar tis ma siz ▁en ▁iyi si ▁ , ▁12 ▁ yi l ▁boyunca ▁ac aba ▁ikincisi ▁çek ir imi ▁diye ▁bekledi gi m ▁bir ▁film ▁ , bel ki ▁william ▁wallace ▁baba sinin ▁ölümünden ▁sonra ▁amca si ▁yani na ▁al m isti ▁onu ▁ ye tis tir m isti"
1,ka dra j lar i yla ▁konusu yla ▁is le ni siyle ▁insana ▁ iste ▁film ▁böyle ▁çekilir ▁de dir ten ▁kusur suz ▁bir ▁film . ▁xxbos ▁böyle ▁güzel ▁bir ▁yap it ▁olamaz ▁filmde ▁her ▁sey ▁var ▁insani ▁dünya dan ▁ali p ▁gö tür üyor ▁bask a ▁diyar lara ▁film ▁bitti kten ▁sonra ▁epey ▁süre ▁geçmesi ▁gerekiyor ▁tekrar ▁dünya ▁ya ▁dönmek ▁için ▁dikkat ! . ▁xxbos ▁inan ir mi siniz
2,▁o ▁kadar ▁etki len mis tim ▁ki . . . özellikle ▁özgürlük ▁ ug ru na ▁sava san ▁william ▁wallace  in ▁is ken ce ▁edilerek ▁idam ▁edilmesi . . . ve ▁sonunda ▁özgürlük ▁diye ▁hay kir isi . . . ha lan ▁unut ami yorum ▁xxrep ▁4 ▁ . ▁xxbos ▁ilk ▁bu ▁filmi ▁sinema da ▁izledi m ▁ve ▁insan in ▁inan di ktan ▁sonra ▁ ne leri ▁yap a
3,▁için ▁ister ▁istemez ▁oraya ▁gö tür üyor ▁filmin ▁uzun lu gun a ▁al dan ip ta ▁filmi ▁izlemek ten ▁vazgeçme yin ▁xxrep ▁4 ▁ . ▁xxbos ▁harika ▁bir ▁film di ▁xxrep ▁5 ▁ . ▁xxbos ▁mükemmel ▁ötesi . . ▁ . ▁xxbos ▁hiç ▁ a bart mi yorum ▁hayat im da ▁izledi gi m ▁en ▁iyi ▁film lerden ▁biri ▁diye bilir im . tam ▁bir ▁bas yap it ▁nite ligi
4,les iyor . ▁herkes ▁izleme li . . . ▁xxbos ▁tarantino nun ▁en ▁iyi ▁filmlerinden ▁biri . ▁diyalog lar ▁çok ▁iyi . ▁kesinlikle ▁izlenmesi ▁gereken ▁bir ▁film . . ▁xxbos ▁tarantino nun ▁bu ▁filmi ▁kendini ▁belli ▁etti r iyor . hat ta ▁ben ce ▁tarantino nun ▁en ▁iyi ▁filmidir . kendi ne ▁has ▁anlat imi ▁ile ▁bu ▁film ▁hak ka ten ▁sinema ▁sever lerin ▁izleme si ▁gereken ▁bir ▁film .


In [31]:
learn_lm = language_model_learner(data_lm, AWD_LSTM, pretrained_fnames=lm_fns, drop_mult=1.0, wd=0.1)

In [32]:
lr = 1e-3
lr *= bs/48

In [33]:
learn_lm.fit_one_cycle(1, lr*10, moms=(0.8,0.7))

epoch,train_loss,valid_loss,time
0,4.762205,4.132825,00:07


In [34]:
learn_lm.unfreeze()
learn_lm.fit_one_cycle(5, slice(lr/10,lr*10), moms=(0.8,0.7))

epoch,train_loss,valid_loss,time
0,4.136730,3.976010,00:08
1,3.991336,3.882426,00:08
2,3.768218,3.812365,00:08
3,3.526307,3.795765,00:08
4,3.310204,3.815228,00:08


In [35]:
learn_lm.save(f'{lang}fine_tuned')
learn_lm.save_encoder(f'{lang}fine_tuned_enc')

### Classifier

In [37]:
data_clas = (TextList.from_df(df, path_clas, cols='text', processor=SPProcessor.load(dest))
    .split_by_rand_pct(0.1, seed=42)
    .label_from_df(cols='pos')
    .databunch(bs=bs, num_workers=1))

In [38]:
learn_c = text_classifier_learner(data_clas, AWD_LSTM, drop_mult=0.5, pretrained=False, wd=0.1).to_fp16()
learn_c.load_encoder(f'{lang}fine_tuned_enc')
learn_c.freeze()

In [39]:
lr=2e-2
lr *= bs/48

In [40]:
learn_c.fit_one_cycle(2, lr, moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,time
0,0.456195,0.441739,0.817073,00:02
1,0.416383,0.359390,0.841463,00:02


In [41]:
learn_c.fit_one_cycle(2, lr, moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,time
0,0.413642,0.381773,0.828330,00:02
1,0.407912,0.358319,0.846154,00:02


In [42]:
learn_c.freeze_to(-2)
learn_c.fit_one_cycle(2, slice(lr/(2.6**4),lr), moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,time
0,0.358337,0.310041,0.869606,00:02
1,0.279921,0.276278,0.892120,00:02


In [43]:
learn_c.freeze_to(-3)
learn_c.fit_one_cycle(2, slice(lr/2/(2.6**4),lr/2), moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,time
0,0.258410,0.261940,0.899625,00:03
1,0.183993,0.267610,0.905253,00:02


In [ ]:
learn_c.unfreeze()
learn_c.fit_one_cycle(4, slice(lr/10/(2.6**4),lr/10), moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,time
0,0.115543,0.290134,0.902439,00:03
1,0.107257,0.297127,0.911820,00:03


Accuracy in Gezici (2018), *Sentiment Analysis in Turkish* is: `75.16%`.

In [158]:
learn_c.save(f'{lang}clas')

## fin